# My First Convolutional Neural Network

## Fashion MNIST

#### 실습목표<br>
1. CNN의 기본 아이디어를 안다.
2. CNN의 구조를 그리고, 코드로 옮길 수 있다.

## Real Game : CNN on Fashion MNIST

여기에서는 여러분이 직접 코드를 완성해야 하는 문제가 곳곳에 있습니다!

In [25]:
'''
matplolib inline 명령어를 통해서
matplot으로 그리는 플롯들을 주피터 노트북 내에서 볼 수 있게 해준다.
포맷을 retina로 바꾸면 그래프의 화질이 훨씬 좋아진다.
'''
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras import layers, models
from tensorflow.keras import backend

import random as rd
from sklearn.metrics import accuracy_score

### Data Loading

In [26]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()

In [27]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [28]:
labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

print(labels)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


id = 3489
다음 그림은 Pullover 입니다.


In [29]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print(f'id = {id}')
print(f'다음 그림은 {labels[test_y[id]]} 입니다.')
plt.imshow(test_x[id], cmap='Greys')
plt.show()

### X : Min-Max Scaling

1. 최소값 0, 최대값 1로 통일하는 스케일링`

In [30]:
train_x = train_x/train_x.max()
test_x = test_x/test_x.max()

### X : Reshape(# of data, 28, 28, 1)

**끝에 1을 달아서 그레이스케일(흑백)을 명시해준다.**

In [31]:
train_x.shape, train_y.shape

((60000, 28, 28), (60000,))

In [32]:
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)

### Y : One-Hot Encoding

In [33]:
from tensorflow.keras.utils import to_categorical

class_n = len(np.unique(train_y))

train_y = to_categorical(train_y, class_n)
test_y = to_categorical(test_y, class_n)

### 코드를 완성해주세요!

**자유롭게 먼저 해보는 것을 추천**




**구조를 따라서 코딩을 한다면..**

1. 인풋레이어
1. Convolution : 필터수 32개, 사이즈(3, 3), same padding
2. BatchNormalization
2. Convolution : 필터수 32개, 사이즈(3, 3), same padding
3. BatchNormalization
4. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
5. DropOut : 25% 비활성화
1. Convolution : 필터수 64개, 사이즈(3, 3), same padding
3. BatchNormalization
2. Convolution : 필터수 64개, 사이즈(3, 3), same padding
3. BatchNormalization
4. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
5. DropOut : 25% 비활성화
6. Flatten()
7. Fully Connected Layer : 노드 512개
3. BatchNormalization
9. 아웃풋레이어

In [34]:
from matplotlib.cbook import flatten
# 1. 세션 클리어
keras.backend.clear_session()

# 2. 모델 생성
model = keras.models.Sequential()

# 3. 레이어 추가
model.add(keras.layers.Input(shape=(28,28,1)))

model.add(keras.layers.Conv2D(filters=32,
                              kernel_size=(3,3),
                              padding='same',
                              activation='relu'))
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(filters=32,
                              kernel_size=(3,3),
                              padding='same',
                              activation='relu'))
model.add(BatchNormalization())

model.add(keras.layers.MaxPool2D(pool_size=(2,2),
                                 strides=(2,2)))
model.add(Dropout(0.25))

model.add(keras.layers.Conv2D(filters=64,
                              kernel_size=(3,3),
                              padding='same',
                              activation='relu'))
model.add(BatchNormalization())
model.add(keras.layers.Conv2D(filters=64,
                              kernel_size=(3,3),
                              padding='same',
                              activation='relu'))
model.add(BatchNormalization())

model.add(keras.layers.MaxPool2D(pool_size=(2,2),
                                 strides=(2,2)))
model.add(Dropout(0.25))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512))
model.add(BatchNormalization())

model.add(keras.layers.Dense(10, activation='softmax'))

# 4. 모델 컴파일
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# 5. 모델 구조 확인
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                        

### Early Stopping을 활용한 학습

1. validation_split = 0.2
2. 1 epochs만 관찰해가며 속도가 가장 빠른 batch_size 찾아보기. 128개부터 시작하여 조절해볼 것.
3. EarlyStopping. val_loss가 4 epoch 전과 비교하여 개선되지 않으면 스탑


In [38]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=4,
                   verbose=1,
                   restore_best_weights=True)

model.fit(train_x, train_y, validation_split=0.2, epochs=1000, batch_size=128, verbose=1, callbacks=[es])

Epoch 1/1000
375/375 [==============================] - 5s 14ms/step - loss: 0.1858 - accuracy: 0.9336 - val_loss: 0.2009 - val_accuracy: 0.9271
Epoch 2/1000
375/375 [==============================] - 5s 14ms/step - loss: 0.1776 - accuracy: 0.9353 - val_loss: 0.2271 - val_accuracy: 0.9179
Epoch 3/1000
375/375 [==============================] - 6s 15ms/step - loss: 0.1750 - accuracy: 0.9371 - val_loss: 0.2026 - val_accuracy: 0.9287
Epoch 4/1000
375/375 [==============================] - 5s 14ms/step - loss: 0.1647 - accuracy: 0.9402 - val_loss: 0.1885 - val_accuracy: 0.9327
Epoch 5/1000
375/375 [==============================] - 5s 14ms/step - loss: 0.1606 - accuracy: 0.9411 - val_loss: 0.1903 - val_accuracy: 0.9332
Epoch 6/1000
375/375 [==============================] - 5s 14ms/step - loss: 0.1535 - accuracy: 0.9435 - val_loss: 0.2120 - val_accuracy: 0.9258
Epoch 7/1000
375/375 [==============================] - 5s 13ms/step - loss: 0.1513 - accuracy: 0.9455 - val_loss: 0.2071 - val_ac

In [ ]:
model.fit(train_x, train_y, validation_split=0.2, epochs=100, verbose=1, callbacks=[es])

<b>* Binary Cross Entropy</b>  
Binary Cross Entropy 는 multi-label classification에서 사용하지만  
 2-class(true, false 혹은 0, 1 등등)의 제한적인 형태에서 사용된다.   
모델에서 정의할 때 마지막에 dense를 2로 해주어야 한다.   
그렇지 않으면 에러가 뜨니까 꼭 모델 만들 때 모든 레이어를 쌓고   
마지막에 두 가지 갈래로 나오게끔 만들면 된다.

<b>* Categorical Cross Entropy</b>  
Categorical Cross Entropy는 데이터 label이 원-핫 인코딩 방식일 때 사용한다.  
 우리가 to_categorical을 죽어라 사용하는 것도 이것 때문일 확률이 높다.

<b>* Sparse Categorical Cross Entropy</b>  
Sparse Categorical Cross Entropy는 반대로 label이 정수일 때 사용한다.   
기본적으로 데이터 셋이 제공될 때 label이 정수 형태를 띄고 있는 경우가 많은데  
 이럴 때 이 loss function을 사용한다.